In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5623,2020-11-17T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1505,227,1732,29024,...,991,1698,12964,971,27716.0,16975.0,44691,835534,577818.0,NaN
5624,2020-11-17T17:00:00,ITA,9,Toscana,43.769231,11.255889,1792,277,2069,52083,...,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0,effettuati 48180 tamponi rapidi antigenici di...
5625,2020-11-17T17:00:00,ITA,10,Umbria,43.106758,12.388247,368,70,438,10806,...,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0,Si fa presente come nel computo dei ricoverati...
5626,2020-11-17T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,152,17,169,1994,...,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5623,2020-11-17T17:00:00,19,Sicilia,1505,227,1732,29024,30756,991,1698,12964,971,27716.0,16975.0,44691,835534,577818.0
5624,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0
5625,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0
5626,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-17', '2020-11-16')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0
1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0
2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0
3,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0
4,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0
5,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0
6,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0
7,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0
8,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0
9,2020-11-17T17:00:00,11,Marche,525,78,603,13908,14511,300,357,8314,1130,23955.0,0.0,23955,372822,218598.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0
1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0
2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0
3,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0
4,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0
5,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0
6,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0
7,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0
8,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0
9,2020-11-16T17:00:00,11,Marche,499,78,577,13634,14211,162,282,8268,1119,23400.0,198.0,23598,370514,217369.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0
1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0
2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0
3,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,16478,956,24981.0,6148.0,31129,612579,NaN
4,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0
5,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0
6,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0
7,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0
8,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0
9,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-16T17:00:00,1,Piemonte,5074,378,5452,67488,72940,1711,3476,50884,5117,53935.0,75006.0,128941,1288923,815862.0
1,1,2020-11-16T17:00:00,2,Valle d'Aosta,151,16,167,2099,2266,-29,99,2826,253,4738.0,607.0,5345,50819,29915.0
2,2,2020-11-16T17:00:00,3,Lombardia,7901,855,8756,143583,152339,-10202,4128,153103,19466,245017.0,79891.0,324908,3568531,2160155.0
3,19,2020-11-16T17:00:00,4,Trentino-Alto Adige,873,75,948,12458,13406,154,725,16245,944,24646.0,5949.0,30595,609621,NaN
4,3,2020-11-16T17:00:00,5,Veneto,1985,252,2237,60461,62698,1192,1966,37277,2867,27639.0,75203.0,102842,2571888,993185.0
5,4,2020-11-16T17:00:00,6,Friuli Venezia Giulia,407,47,454,9601,10055,128,456,8792,546,16657.0,2736.0,19393,623585,254630.0
6,5,2020-11-16T17:00:00,7,Liguria,1381,111,1492,14409,15901,163,365,25717,2124,30461.0,13281.0,43742,529552,266634.0
7,6,2020-11-16T17:00:00,8,Emilia-Romagna,2393,247,2640,52789,55429,2228,2547,30570,5067,58909.0,32157.0,91066,1873435,980006.0
8,7,2020-11-16T17:00:00,9,Toscana,1777,284,2061,51790,53851,1543,2433,26070,1915,65242.0,16594.0,81836,1347451,863173.0
9,8,2020-11-16T17:00:00,10,Umbria,375,72,447,10792,11239,144,283,7658,262,5348.0,13811.0,19159,361474,203887.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0,4341375
1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0,125501
2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0,10103969
3,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,16478,956,24981.0,6148.0,31129,612579,NaN,1074819
4,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0,4907704
5,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0,1211357
6,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0,1543127
7,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0,4467118
8,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0,3722729
9,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,52571,5190,55023.0,76524.0,131547,1305054,826436.0,4341375,575
1,2020-11-17T17:00:00,2,Valle d'Aosta,152,17,169,1994,2163,-103,154,3079,257,4884.0,615.0,5499,51410,30178.0,125501,20
2,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,157121,19668,251767.0,81589.0,333356,3606814,2177539.0,10103969,1036
3,2020-11-17T17:00:00,4,Trentino-Alto Adige,870,79,949,12746,13695,289,534,16478,956,24981.0,6148.0,31129,612579,NaN,1074819,106
4,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,39928,2967,27936.0,78030.0,105966,2590427,1000577.0,4907704,1016
5,2020-11-17T17:00:00,6,Friuli Venezia Giulia,454,44,498,9923,10421,366,536,8952,556,17057.0,2872.0,19929,629511,255770.0,1211357,180
6,2020-11-17T17:00:00,7,Liguria,1393,117,1510,15021,16531,630,685,25752,2144,30766.0,13661.0,44427,535222,268832.0,1543127,209
7,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,30902,5115,60144.0,33141.0,93285,1895816,988202.0,4467118,634
8,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,28078,1967,66908.0,17289.0,84197,1363146,870134.0,3722729,523
9,2020-11-17T17:00:00,10,Umbria,368,70,438,10806,11244,5,351,7991,275,5465.0,14045.0,19510,367077,206060.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-17T17:00:00,3,Lombardia,8151,894,9045,147522,156567,4228,8448,...,86.29,3939,4320,38283,17384.0,202,4018,22.07,0.08361,3.30
1,2020-11-17T17:00:00,5,Veneto,1980,266,2246,60825,63071,373,3124,...,26.18,364,1158,18539,7392.0,100,2651,16.85,0.06366,2.16
2,2020-11-17T17:00:00,15,Campania,2236,201,2437,90289,92726,1212,3019,...,39.80,1157,-1060,16178,12734.0,19,1788,18.66,0.05218,2.04
3,2020-11-17T17:00:00,1,Piemonte,5150,384,5534,68252,73786,846,2606,...,66.78,764,-870,16131,10574.0,73,1687,16.16,0.06003,3.03
4,2020-11-17T17:00:00,12,Lazio,3066,308,3374,67881,71255,1985,2538,...,36.36,1988,131,25058,21008.0,59,494,10.13,0.04327,1.52
5,2020-11-17T17:00:00,9,Toscana,1792,277,2069,52083,54152,301,2361,...,52.96,293,-72,15695,6961.0,52,2008,15.04,0.06342,2.26
6,2020-11-17T17:00:00,8,Emilia-Romagna,2451,247,2698,54570,57268,1839,2219,...,38.96,1781,-328,22381,8196.0,48,332,9.91,0.04967,2.09
7,2020-11-17T17:00:00,19,Sicilia,1505,227,1732,29024,30756,991,1698,...,38.61,984,237,10774,7001.0,39,668,15.76,0.03418,0.90
8,2020-11-17T17:00:00,16,Puglia,1316,181,1497,25110,26607,1082,1234,...,49.05,1062,190,6228,5638.0,36,116,19.81,0.03079,0.92
9,2020-11-17T17:00:00,13,Abruzzo,590,66,656,13352,14008,509,729,...,44.59,483,87,4953,2326.0,13,207,14.72,0.05583,1.57


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,894,142,1036,86.29
1,Valle d'Aosta,17,3,20,85.00
2,Trentino-Alto Adige,79,27,106,74.53
3,Piemonte,384,191,575,66.78
4,Umbria,70,47,117,59.83
5,Liguria,117,92,209,55.98
6,Marche,78,65,143,54.55
7,Toscana,277,246,523,52.96
8,Puglia,181,188,369,49.05
9,Abruzzo,66,82,148,44.59


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
